In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    #     aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
    #                                                                       labels=actions_labels))
    neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                              labels=actions_labels)
    ################################################ s', r = env(s, a)
    next_states_logits, Qs_logits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                        action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits,
                                                                   labels=next_states_labels))
    #################################################### s'', Q' = env(s', a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_next_states_logits, nextQs_logits = Env(actions=next_actions_logits, states=next_states, # given
                                                 hidden_size=hidden_size, action_size=action_size, 
                                                 state_size=state_size, reuse=True)
    #################################################### s'', Q' = env(~s', ~a')
    next_actions_logits_ = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                               reuse=True)
    next_next_states_logits_, nextQs_logits_ = Env(actions=next_actions_logits_, states=next_states_logits, # pred
                                                   hidden_size=hidden_size, action_size=action_size, 
                                                   state_size=state_size, reuse=True)
    ##################################################### GAN
    Qs = tf.reshape(Qs_logits, shape=[-1])
    nextQs = tf.reshape(nextQs_logits, shape=[-1]) * (1-dones)
    nextQs_ = tf.reshape(nextQs_logits_, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs, # GAN
                                                                    labels=rates)) # 0-1 real
    #     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(nextQs+nextQs_)/2, # GAN
    #                                                                     labels=tf.zeros_like(rates))) # min
    #     aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(nextQs+nextQs_)/2, # GAN
    #                                                                     labels=tf.ones_like(rates))) # max
    #     ########################################################## Qlearning
    #     targetQs = rewards + (nextQs+nextQs_)/2
    #     eloss += tf.reduce_mean(tf.square(Qs - targetQs))
    ######################################################### Curosity=intrinsic reward
    rloss = tf.square(nextQs-nextQs_)
    sloss = tf.reduce_sum(tf.square(next_next_states_logits-next_next_states_logits_), axis=1)
    #### Q/r loss
    mean, var = tf.nn.moments(x=rloss,axes=[0])
    std = tf.sqrt(var)
    rloss_norm = (rloss - mean)/ std 
    #### State loss
    mean, var = tf.nn.moments(x=sloss,axes=[0])
    std = tf.sqrt(var)
    sloss_norm = (sloss - mean)/ std
    #### Intrinsic_rewards
    intrinsic_rewards = sloss_norm + rloss_norm
    aloss = tf.reduce_mean(neg_log_prob*intrinsic_rewards)
    return actions_logits, aloss, eloss

In [7]:
def model_opt(a_loss, e_loss, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
    return a_opt, e_opt

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt = model_opt(a_loss=self.a_loss, e_loss=self.e_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch= [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:0.0606 eloss:1.4280 exploreP:0.9983
Episode:1 meanR:24.0000 R:31.0000 rate:0.0620 aloss:0.0543 eloss:1.4155 exploreP:0.9953
Episode:2 meanR:21.6667 R:17.0000 rate:0.0340 aloss:0.0124 eloss:1.4119 exploreP:0.9936
Episode:3 meanR:24.5000 R:33.0000 rate:0.0660 aloss:0.0010 eloss:1.3976 exploreP:0.9903
Episode:4 meanR:26.0000 R:32.0000 rate:0.0640 aloss:-0.0153 eloss:1.3875 exploreP:0.9872
Episode:5 meanR:23.6667 R:12.0000 rate:0.0240 aloss:-0.0831 eloss:1.3985 exploreP:0.9860
Episode:6 meanR:23.4286 R:22.0000 rate:0.0440 aloss:-0.0057 eloss:1.3858 exploreP:0.9839
Episode:7 meanR:25.0000 R:36.0000 rate:0.0720 aloss:-0.0101 eloss:1.3740 exploreP:0.9804
Episode:8 meanR:27.8889 R:51.0000 rate:0.1020 aloss:-0.0378 eloss:1.3617 exploreP:0.9755
Episode:9 meanR:26.9000 R:18.0000 rate:0.0360 aloss:-0.0533 eloss:1.3488 exploreP:0.9737
Episode:10 meanR:25.5455 R:12.0000 rate:0.0240 aloss:-0.0307 eloss:1.3544 exploreP:0.9726
Episode:11 meanR:24.5833

Episode:92 meanR:20.9570 R:24.0000 rate:0.0480 aloss:-2.4995 eloss:1.1637 exploreP:0.8247
Episode:93 meanR:21.0213 R:27.0000 rate:0.0540 aloss:-2.6297 eloss:1.1736 exploreP:0.8225
Episode:94 meanR:20.9053 R:10.0000 rate:0.0200 aloss:-2.8450 eloss:1.1497 exploreP:0.8217
Episode:95 meanR:20.8854 R:19.0000 rate:0.0380 aloss:-2.8188 eloss:1.1646 exploreP:0.8201
Episode:96 meanR:20.8660 R:19.0000 rate:0.0380 aloss:-2.8176 eloss:1.1693 exploreP:0.8186
Episode:97 meanR:20.8673 R:21.0000 rate:0.0420 aloss:-2.9918 eloss:1.1641 exploreP:0.8169
Episode:98 meanR:20.7778 R:12.0000 rate:0.0240 aloss:-3.1125 eloss:1.1748 exploreP:0.8159
Episode:99 meanR:20.7100 R:14.0000 rate:0.0280 aloss:-3.2444 eloss:1.1629 exploreP:0.8148
Episode:100 meanR:20.8600 R:32.0000 rate:0.0640 aloss:-3.2460 eloss:1.1621 exploreP:0.8122
Episode:101 meanR:20.7200 R:17.0000 rate:0.0340 aloss:-3.0009 eloss:1.1768 exploreP:0.8109
Episode:102 meanR:20.7800 R:23.0000 rate:0.0460 aloss:-3.3591 eloss:1.1612 exploreP:0.8090
Episode

Episode:183 meanR:18.8200 R:11.0000 rate:0.0220 aloss:-10.7942 eloss:1.1379 exploreP:0.6975
Episode:184 meanR:18.7500 R:13.0000 rate:0.0260 aloss:-11.6211 eloss:1.1376 exploreP:0.6966
Episode:185 meanR:18.7000 R:16.0000 rate:0.0320 aloss:-11.7150 eloss:1.1338 exploreP:0.6955
Episode:186 meanR:18.7000 R:13.0000 rate:0.0260 aloss:-12.0609 eloss:1.1404 exploreP:0.6946
Episode:187 meanR:18.4100 R:12.0000 rate:0.0240 aloss:-12.4017 eloss:1.1555 exploreP:0.6938
Episode:188 meanR:18.3000 R:19.0000 rate:0.0380 aloss:-12.0544 eloss:1.1359 exploreP:0.6925
Episode:189 meanR:18.3300 R:15.0000 rate:0.0300 aloss:-11.3471 eloss:1.1359 exploreP:0.6915
Episode:190 meanR:18.3400 R:11.0000 rate:0.0220 aloss:-13.0889 eloss:1.1476 exploreP:0.6908
Episode:191 meanR:18.3400 R:14.0000 rate:0.0280 aloss:-12.5927 eloss:1.1354 exploreP:0.6898
Episode:192 meanR:18.2200 R:12.0000 rate:0.0240 aloss:-12.9594 eloss:1.1312 exploreP:0.6890
Episode:193 meanR:18.1000 R:15.0000 rate:0.0300 aloss:-13.1039 eloss:1.1586 expl

Episode:273 meanR:15.5300 R:12.0000 rate:0.0240 aloss:-27.3397 eloss:1.1502 exploreP:0.6071
Episode:274 meanR:15.5400 R:11.0000 rate:0.0220 aloss:-27.7164 eloss:1.1411 exploreP:0.6065
Episode:275 meanR:15.5500 R:11.0000 rate:0.0220 aloss:-28.7838 eloss:1.1190 exploreP:0.6058
Episode:276 meanR:15.7300 R:38.0000 rate:0.0760 aloss:-28.5229 eloss:1.1316 exploreP:0.6035
Episode:277 meanR:15.7400 R:12.0000 rate:0.0240 aloss:-28.9973 eloss:1.1272 exploreP:0.6028
Episode:278 meanR:15.7700 R:25.0000 rate:0.0500 aloss:-27.7983 eloss:1.1229 exploreP:0.6013
Episode:279 meanR:15.7500 R:12.0000 rate:0.0240 aloss:-27.5072 eloss:1.1183 exploreP:0.6006
Episode:280 meanR:15.6400 R:9.0000 rate:0.0180 aloss:-29.0967 eloss:1.1068 exploreP:0.6001
Episode:281 meanR:15.6700 R:14.0000 rate:0.0280 aloss:-27.5125 eloss:1.1736 exploreP:0.5993
Episode:282 meanR:15.6400 R:11.0000 rate:0.0220 aloss:-24.0758 eloss:1.1418 exploreP:0.5986
Episode:283 meanR:15.6200 R:9.0000 rate:0.0180 aloss:-29.9844 eloss:1.1199 explor

Episode:363 meanR:14.9900 R:12.0000 rate:0.0240 aloss:-54.0887 eloss:1.1122 exploreP:0.5336
Episode:364 meanR:15.0100 R:13.0000 rate:0.0260 aloss:-54.7963 eloss:1.1146 exploreP:0.5330
Episode:365 meanR:14.8100 R:10.0000 rate:0.0200 aloss:-55.4641 eloss:1.1104 exploreP:0.5324
Episode:366 meanR:14.9100 R:25.0000 rate:0.0500 aloss:-58.1352 eloss:1.1143 exploreP:0.5311
Episode:367 meanR:14.8100 R:14.0000 rate:0.0280 aloss:-46.6375 eloss:1.0997 exploreP:0.5304
Episode:368 meanR:14.8400 R:12.0000 rate:0.0240 aloss:-55.6896 eloss:1.1049 exploreP:0.5298
Episode:369 meanR:14.9000 R:18.0000 rate:0.0360 aloss:-55.9475 eloss:1.1136 exploreP:0.5288
Episode:370 meanR:14.5700 R:11.0000 rate:0.0220 aloss:-55.5372 eloss:1.1051 exploreP:0.5283
Episode:371 meanR:14.5100 R:10.0000 rate:0.0200 aloss:-63.2031 eloss:1.1210 exploreP:0.5278
Episode:372 meanR:14.6400 R:26.0000 rate:0.0520 aloss:-58.4990 eloss:1.1132 exploreP:0.5264
Episode:373 meanR:14.6100 R:9.0000 rate:0.0180 aloss:-61.4787 eloss:1.1179 explo

...

Episode:898 meanR:11.0700 R:9.0000 rate:0.0180 aloss:-371.0672 eloss:1.0756 exploreP:0.2825
Episode:899 meanR:11.0500 R:8.0000 rate:0.0160 aloss:-401.7111 eloss:1.0469 exploreP:0.2823
Episode:900 meanR:11.0200 R:10.0000 rate:0.0200 aloss:-381.4991 eloss:1.0719 exploreP:0.2820
Episode:901 meanR:11.0200 R:9.0000 rate:0.0180 aloss:-372.3199 eloss:1.0447 exploreP:0.2818
Episode:902 meanR:11.0100 R:9.0000 rate:0.0180 aloss:-349.8642 eloss:1.0429 exploreP:0.2815
Episode:903 meanR:11.0700 R:17.0000 rate:0.0340 aloss:-382.3184 eloss:1.0517 exploreP:0.2811
Episode:904 meanR:11.0700 R:10.0000 rate:0.0200 aloss:-378.3493 eloss:1.0053 exploreP:0.2808
Episode:905 meanR:11.1200 R:14.0000 rate:0.0280 aloss:-370.9478 eloss:1.0704 exploreP:0.2804
Episode:906 meanR:11.1000 R:10.0000 rate:0.0200 aloss:-415.5637 eloss:1.0200 exploreP:0.2802
Episode:907 meanR:11.0400 R:11.0000 rate:0.0220 aloss:-405.3162 eloss:1.0340 exploreP:0.2799
Episode:908 meanR:11.0300 R:10.0000 rate:0.0200 aloss:-393.8879 eloss:1.07

Episode:987 meanR:10.8400 R:9.0000 rate:0.0180 aloss:-526.5364 eloss:1.0737 exploreP:0.2571
Episode:988 meanR:10.8000 R:10.0000 rate:0.0200 aloss:-507.4831 eloss:1.0573 exploreP:0.2569
Episode:989 meanR:10.7900 R:11.0000 rate:0.0220 aloss:-477.9676 eloss:1.0113 exploreP:0.2566
Episode:990 meanR:10.8200 R:12.0000 rate:0.0240 aloss:-524.6293 eloss:1.0393 exploreP:0.2563
Episode:991 meanR:10.8200 R:8.0000 rate:0.0160 aloss:-463.3313 eloss:1.0778 exploreP:0.2561
Episode:992 meanR:10.8400 R:10.0000 rate:0.0200 aloss:-409.4246 eloss:1.0547 exploreP:0.2559
Episode:993 meanR:10.8500 R:9.0000 rate:0.0180 aloss:-473.1004 eloss:1.0401 exploreP:0.2556
Episode:994 meanR:10.8600 R:10.0000 rate:0.0200 aloss:-561.3896 eloss:1.0629 exploreP:0.2554
Episode:995 meanR:10.8400 R:11.0000 rate:0.0220 aloss:-551.5921 eloss:1.0092 exploreP:0.2551
Episode:996 meanR:10.8700 R:11.0000 rate:0.0220 aloss:-524.3141 eloss:1.0551 exploreP:0.2549
Episode:997 meanR:10.8900 R:10.0000 rate:0.0200 aloss:-535.5656 eloss:1.0

Episode:1075 meanR:10.7200 R:9.0000 rate:0.0180 aloss:-597.0501 eloss:1.0501 exploreP:0.2349
Episode:1076 meanR:10.7500 R:14.0000 rate:0.0280 aloss:-638.6479 eloss:1.0448 exploreP:0.2346
Episode:1077 meanR:10.7300 R:11.0000 rate:0.0220 aloss:-597.9781 eloss:1.0648 exploreP:0.2343
Episode:1078 meanR:10.7600 R:15.0000 rate:0.0300 aloss:-558.7618 eloss:1.0416 exploreP:0.2340
Episode:1079 meanR:10.7400 R:10.0000 rate:0.0200 aloss:-599.0745 eloss:1.0237 exploreP:0.2338
Episode:1080 meanR:10.7200 R:9.0000 rate:0.0180 aloss:-567.8040 eloss:1.0693 exploreP:0.2336
Episode:1081 meanR:10.7200 R:9.0000 rate:0.0180 aloss:-604.7771 eloss:1.0570 exploreP:0.2334
Episode:1082 meanR:10.6900 R:8.0000 rate:0.0160 aloss:-608.6433 eloss:1.0511 exploreP:0.2332
Episode:1083 meanR:10.7300 R:16.0000 rate:0.0320 aloss:-613.3496 eloss:1.0227 exploreP:0.2329
Episode:1084 meanR:10.7400 R:11.0000 rate:0.0220 aloss:-503.9615 eloss:1.0146 exploreP:0.2326
Episode:1085 meanR:10.7400 R:9.0000 rate:0.0180 aloss:-640.7518 

Episode:1163 meanR:10.4700 R:8.0000 rate:0.0160 aloss:-663.4512 eloss:1.0903 exploreP:0.2151
Episode:1164 meanR:10.4800 R:11.0000 rate:0.0220 aloss:-677.8510 eloss:1.0242 exploreP:0.2148
Episode:1165 meanR:10.4200 R:9.0000 rate:0.0180 aloss:-628.9369 eloss:1.0583 exploreP:0.2147
Episode:1166 meanR:10.4300 R:10.0000 rate:0.0200 aloss:-599.5435 eloss:1.0432 exploreP:0.2144
Episode:1167 meanR:10.4300 R:10.0000 rate:0.0200 aloss:-585.2939 eloss:1.0751 exploreP:0.2142
Episode:1168 meanR:10.4100 R:9.0000 rate:0.0180 aloss:-426.1130 eloss:1.0364 exploreP:0.2141
Episode:1169 meanR:10.3500 R:9.0000 rate:0.0180 aloss:-621.1497 eloss:1.0190 exploreP:0.2139
Episode:1170 meanR:10.3500 R:9.0000 rate:0.0180 aloss:-646.6882 eloss:1.0369 exploreP:0.2137
Episode:1171 meanR:10.3800 R:12.0000 rate:0.0240 aloss:-544.9570 eloss:1.0397 exploreP:0.2134
Episode:1172 meanR:10.4000 R:11.0000 rate:0.0220 aloss:-550.1936 eloss:1.0673 exploreP:0.2132
Episode:1173 meanR:10.4000 R:9.0000 rate:0.0180 aloss:-422.0887 e

Episode:1251 meanR:10.5100 R:9.0000 rate:0.0180 aloss:-648.5220 eloss:1.0472 exploreP:0.1969
Episode:1252 meanR:10.5200 R:10.0000 rate:0.0200 aloss:-515.9316 eloss:1.0501 exploreP:0.1967
Episode:1253 meanR:10.5000 R:9.0000 rate:0.0180 aloss:-738.3124 eloss:1.0056 exploreP:0.1966
Episode:1254 meanR:10.5100 R:12.0000 rate:0.0240 aloss:-495.1057 eloss:1.0351 exploreP:0.1963
Episode:1255 meanR:10.5100 R:8.0000 rate:0.0160 aloss:-627.6126 eloss:1.0531 exploreP:0.1962
Episode:1256 meanR:10.4800 R:9.0000 rate:0.0180 aloss:-660.5451 eloss:1.0587 exploreP:0.1960
Episode:1257 meanR:10.4900 R:11.0000 rate:0.0220 aloss:-800.4637 eloss:1.0055 exploreP:0.1958
Episode:1258 meanR:10.4900 R:10.0000 rate:0.0200 aloss:-639.9122 eloss:1.0300 exploreP:0.1956
Episode:1259 meanR:10.4400 R:8.0000 rate:0.0160 aloss:-722.0903 eloss:1.0269 exploreP:0.1955
Episode:1260 meanR:10.4800 R:14.0000 rate:0.0280 aloss:-796.9644 eloss:1.0454 exploreP:0.1952
Episode:1261 meanR:10.4900 R:12.0000 rate:0.0240 aloss:-663.2708 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
